# Import Data

In [0]:
# connect so s3 bucket
# get credentials
import os

ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
SECRET_KEY = os.getenv("AWS_SECRET_KEY")
ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")
AWS_BUCKET_NAME = "aida-project"
MOUNT_NAME = "data"

# mount data
try:
  dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
except:
  display(dbutils.fs.ls("/mnt/%s" % MOUNT_NAME))

path,name,size
dbfs:/mnt/data/TSV/,TSV/,0
dbfs:/mnt/data/niy/,niy/,0
dbfs:/mnt/data/team_remote/,team_remote/,0


In [0]:
display(dbutils.fs.ls("/mnt/%s/TSV" % MOUNT_NAME))

path,name,size
dbfs:/mnt/data/TSV/name.basics.tsv,name.basics.tsv,579976550
dbfs:/mnt/data/TSV/title.akas.tsv,title.akas.tsv,969441812
dbfs:/mnt/data/TSV/title.basics.tsv,title.basics.tsv,537519832
dbfs:/mnt/data/TSV/title.principals.tsv,title.principals.tsv,1622240736
dbfs:/mnt/data/TSV/title.ratings.tsv,title.ratings.tsv,16907124


In [0]:
df_names = spark.read.load("dbfs:/mnt/data/TSV/name.basics.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_akas = spark.read.load("dbfs:/mnt/data/TSV/title.akas.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_basics = spark.read.load("dbfs:/mnt/data/TSV/title.basics.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_principals = spark.read.load("dbfs:/mnt/data/TSV/title.principals.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_ratings = spark.read.load("dbfs:/mnt/data/TSV/title.ratings.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")

In [0]:
list_dfs = [df_names, df_akas, df_basics, df_principals, df_ratings]

for df in list_dfs:
  df.printSchema()

root
-- nconst: string (nullable = true)
-- primaryName: string (nullable = true)
-- birthYear: string (nullable = true)
-- deathYear: string (nullable = true)
-- primaryProfession: string (nullable = true)
-- knownForTitles: string (nullable = true)

root
-- titleId: string (nullable = true)
-- ordering: integer (nullable = true)
-- title: string (nullable = true)
-- region: string (nullable = true)
-- language: string (nullable = true)
-- types: string (nullable = true)
-- attributes: string (nullable = true)
-- isOriginalTitle: string (nullable = true)

root
-- tconst: string (nullable = true)
-- titleType: string (nullable = true)
-- primaryTitle: string (nullable = true)
-- originalTitle: string (nullable = true)
-- isAdult: integer (nullable = true)
-- startYear: string (nullable = true)
-- endYear: string (nullable = true)
-- runtimeMinutes: string (nullable = true)
-- genres: string (nullable = true)

root
-- tconst: string (nullable = true)
-- ordering: integer (nullable = true)
-- nconst: string (nullable = true)
-- category: string (nullable = true)
-- job: string (nullable = true)
-- characters: string (nullable = true)

root
-- tconst: string (nullable = true)
-- averageRating: double (nullable = true)
-- numVotes: integer (nullable = true)

# Feature Selection & Data Cleaning

In [0]:
from pyspark.sql.functions import mean as _mean, \
                                  min as _min, \
                                  max as _max, \
                                  count as _count, \
                                  stddev as _stddev, \
                                  countDistinct, col, isnan, split, array_distinct, explode

## Sampling

In [0]:
# until final model evaluation use sample

random_state = 42
sample_size = 0.1

train_size = 0.8
test_size = 1 - train_size

def sample(dataframe):
  df_ids = dataframe.select('tconst', 'averageRating').sample(sample_size, random_state)
  return df_ids

## Table: Ratings

In [0]:
def votes(dataframe):
  df_ids = dataframe.join(df_ratings, ['tconst', 'averageRating'])
  return df_ids

## Table: Principals

In [0]:
def principals(dataframe):
  df_a = dataframe.join(df_principals.drop('job','characters'), on='tconst')\
      .groupBy('tconst', 'averageRating', 'numVotes')\
      .agg(_max('ordering').alias('principal_counts'), countDistinct('category').alias('distinct_count_categories'))
  
  df_b = dataframe.join(df_principals, on='tconst').select('tconst', 'category').groupBy('tconst').pivot('category').count().fillna(0)
  
  df_ids = df_a.join(df_b, on='tconst')
  
  return df_ids

## Table: AKAS

In [0]:
from pyspark.sql.functions import split, explode, array_distinct
#df_akas_sample = df_akas.sample(False, 0.000001, 42)
df_akas_sample = df_akas

def akas(dataframe):
  droplist = ['title', 'language', 'types', 'attributes', 'isOriginalTitle']
  df_akas_feature = df_akas_sample.drop(*droplist)
  column = 'region'
  df_akas_feature = df_akas_feature.where(fr'{column} != "\\N"')
  df_akas_pivot = df_akas_feature.groupBy('titleId').pivot('region').count().fillna(0)
  df_akas_pivot = df_akas_pivot.withColumnRenamed('titleId', 'tconst')
  df_ids = dataframe.join(df_akas_pivot, on='tconst')
  return df_ids

## Table: Basics

In [0]:
def basics(dataframe):
  
  droplist = ['primaryTitle', 'originalTitle', 'endYear']
  df_basics_sample_fs = df_basics.drop(*droplist)
  
  df_basics_sample_pivot = df_basics_sample_fs.select('tconst', 'genres', explode(array_distinct(split('genres', ',')))).groupBy('tconst').pivot('col').count().fillna(0)
  df_basics_sample_pivot = df_basics_sample_pivot.drop(r'\N')
                                                                              
  df_ids = dataframe.join(df_basics_sample_pivot, on='tconst')\
     
  return df_ids

# Train-Test Split

In [0]:
df_ids = sample(df_ratings)
df_ids = votes(df_ids)
df_ids = principals(df_ids)
df_ids = akas(df_ids)
df_ids = basics(df_ids)
df_ids.printSchema()

root
-- tconst: string (nullable = true)
-- averageRating: double (nullable = true)
-- numVotes: integer (nullable = true)
-- principal_counts: integer (nullable = true)
-- distinct_count_categories: long (nullable = false)
-- actor: long (nullable = true)
-- actress: long (nullable = true)
-- archive_footage: long (nullable = true)
-- archive_sound: long (nullable = true)
-- cinematographer: long (nullable = true)
-- composer: long (nullable = true)
-- director: long (nullable = true)
-- editor: long (nullable = true)
-- producer: long (nullable = true)
-- production_designer: long (nullable = true)
-- self: long (nullable = true)
-- writer: long (nullable = true)
-- AD: long (nullable = true)
-- AE: long (nullable = true)
-- AF: long (nullable = true)
-- AG: long (nullable = true)
-- AI: long (nullable = true)
-- AL: long (nullable = true)
-- AM: long (nullable = true)
-- AN: long (nullable = true)
-- AO: long (nullable = true)
-- AQ: long (nullable = true)
-- AR: long (nullable = true)
-- AS: long (nullable = true)
-- AT: long (nullable = true)
-- AU: long (nullable = true)
-- AW: long (nullable = true)
-- AZ: long (nullable = true)
-- BA: long (nullable = true)
-- BB: long (nullable = true)
-- BD: long (nullable = true)
-- BE: long (nullable = true)
-- BF: long (nullable = true)
-- BG: long (nullable = true)
-- BH: long (nullable = true)
-- BI: long (nullable = true)
-- BJ: long (nullable = true)
-- BM: long (nullable = true)
-- BN: long (nullable = true)
-- BO: long (nullable = true)
-- BR: long (nullable = true)
-- BS: long (nullable = true)
-- BT: long (nullable = true)
-- BUMM: long (nullable = true)
-- BW: long (nullable = true)
-- BY: long (nullable = true)
-- BZ: long (nullable = true)
-- CA: long (nullable = true)
-- CD: long (nullable = true)
-- CF: long (nullable = true)
-- CG: long (nullable = true)
-- CH: long (nullable = true)
-- CI: long (nullable = true)
-- CK: long (nullable = true)
-- CL: long (nullable = true)
-- CM: long (nullable = true)
-- CN: long (nullable = true)
-- CO: long (nullable = true)
-- CR: long (nullable = true)
-- CSHH: long (nullable = true)
-- CSXX: long (nullable = true)
-- CU: long (nullable = true)
-- CV: long (nullable = true)
-- CY: long (nullable = true)
-- CZ: long (nullable = true)
-- DDDE: long (nullable = true)
-- DE: long (nullable = true)
-- DJ: long (nullable = true)
-- DK: long (nullable = true)
-- DM: long (nullable = true)
-- DO: long (nullable = true)
-- DZ: long (nullable = true)
-- EC: long (nullable = true)
-- EE: long (nullable = true)
-- EG: long (nullable = true)
-- EH: long (nullable = true)
-- ER: long (nullable = true)
-- ES: long (nullable = true)
-- ET: long (nullable = true)
-- FI: long (nullable = true)
-- FJ: long (nullable = true)
-- FO: long (nullable = true)
-- FR: long (nullable = true)
-- GA: long (nullable = true)
-- GB: long (nullable = true)
-- GD: long (nullable = true)
-- GE: long (nullable = true)
-- GF: long (nullable = true)
-- GH: long (nullable = true)
-- GI: long (nullable = true)
-- GL: long (nullable = true)
-- GM: long (nullable = true)
-- GN: long (nullable = true)
-- GP: long (nullable = true)
-- GQ: long (nullable = true)
-- GR: long (nullable = true)
-- GT: long (nullable = true)
-- GU: long (nullable = true)
-- GW: long (nullable = true)
-- GY: long (nullable = true)
-- HK: long (nullable = true)
-- HN: long (nullable = true)
-- HR: long (nullable = true)
-- HT: long (nullable = true)
-- HU: long (nullable = true)
-- ID: long (nullable = true)
-- IE: long (nullable = true)
-- IL: long (nullable = true)
-- IM: long (nullable = true)
-- IN: long (nullable = true)
-- IQ: long (nullable = true)
-- IR: long (nullable = true)
-- IS: long (nullable = true)
-- IT: long (nullable = true)
-- JE: long (nullable = true)
-- JM: long (nullable = true)
-- JO: long (nullable = true)
-- JP: long (nullable = true)
-- KE: long (nullable = true)
-- KG: long (nullable = true)
-- KH: long (nullable = true)
-- KI: long (nullable = true)
-- KM: long (nulla

In [0]:
df_train, df_test = df_ids.randomSplit([train_size, test_size], random_state)

# Write to S3

In [0]:
def save_to_s3(dataframe, label):
  dataframe.drop('tconst') \
    .repartition(1) \
    .write.option("header", "false") \
    .save(f'/mnt/data/niy/{label}', format='csv')
  return "Successfully saved to s3"

In [0]:
############### WRITE TRAINING DATA #################
try:
  save_to_s3(df_train, 'train')
except:
  dbutils.fs.rm('mnt/data/niy/train', True)
  save_to_s3(df_train, 'train')
  
file = dbutils.fs.ls('mnt/data/niy/train')[-1].path
dbutils.fs.cp(file, '/mnt/data/niy/train.csv')
dbutils.fs.rm(file)

Out[113]: True

In [0]:
############### WRITE TESTING DATA #################
try:
  save_to_s3(df_test, 'test')
except:
  dbutils.fs.rm('mnt/data/niy/test', True)
  save_to_s3(df_test, 'test')
  
file = dbutils.fs.ls('mnt/data/niy/test')[-1].path
dbutils.fs.cp(file, '/mnt/data/niy/test.csv')
dbutils.fs.rm(file)

Out[114]: True

In [0]:
# !hadoop fs -getmerge mnt/data/niy/partitioned mnt/data/niy/merged/test.csv